# Notebook 05 – Sistema Multiagente (Nivel Experto)

En este notebook:
- Aprendemos qué es un sistema multiagente
- Diseñamos agentes especializados
- Creamos un router con LLM
- Definimos un orquestador central

Después, este código se copiará al backend.


📁backend/app/models/llms/groq_llm.py

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.3
)

📁backend/app/models/agents/base_agent.py

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from app.models.llms.groq_llm import get_groq_llm

class BaseAgent:
    def __init__(self, chain):
        self.chain = chain

    def invoke(self, input_data: dict) -> str:
        return self.chain.invoke(input_data)


📁backend/app/models/agents/marketing_agent.py

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from app.models.agents.base_agent import BaseAgent
from app.models.llms.groq_llm import get_groq_llm

llm = get_groq_llm()

marketing_prompt = ChatPromptTemplate.from_template("""
Eres un experto creador de contenido digital.

Tema: {tema}
Plataforma: {plataforma}
Audiencia: {audiencia}
Tono: {tono}

Genera contenido listo para publicar.
""")

marketing_chain = marketing_prompt | llm
marketing_agent = BaseAgent(marketing_chain)


📁Backend/app/models/agents/science_agent.py

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from app.models.agents.base_agent import BaseAgent
from app.models.llms.groq_llm import get_groq_llm

llm = get_groq_llm()

science_prompt = ChatPromptTemplate.from_template("""
Eres un divulgador científico experto.

Tema: {tema}

Explica el concepto de forma rigurosa pero accesible
para el público general.
""")

science_chain = science_prompt | llm
science_agent = BaseAgent(science_chain)


📁backend/app/models/agents/router_agent.py


In [ ]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from app.models.llms.groq_llm import get_groq_llm

llm = get_groq_llm()

router_prompt = ChatPromptTemplate.from_template("""
Clasifica la siguiente petición en una categoría:

- marketing
- science

Petición: {input}

Responde SOLO con una palabra.
""")

router_chain = router_prompt | llm

def route_request(text: str) -> Literal["marketing", "science"]:
    return router_chain.invoke({"input": text}).strip().lower()


📁backend/app/services/content_service.py

In [ ]:
from app.models.agents.router_agent import route_request
from app.models.agents.marketing_agent import marketing_agent
from app.models.agents.science_agent import science_agent

def multiagent_generate(input_text: str, payload: dict) -> str:
    route = route_request(input_text)

    if route == "science":
        return science_agent.invoke(payload)

    return marketing_agent.invoke(payload)


In [ ]:
"""
multiagent_generate(
  input_text="Explica la computación cuántica",
  payload={
    "tema": "Computación cuántica",
    "plataforma": "Blog",
    "audiencia": "Público general",
    "tono": "Divulgativo"
  }
)
"""